# Final Project: Regression Analysis of Medical Insurance Charges  

**Author:** Brandon   
**Date:** 2025-11-23  

This project uses regression analysis to model and predict medical insurance charges based on patient characteristics.  
The dataset includes information such as age, sex, BMI, number of children, smoking status, and region.  
The main goal is to understand how these features relate to insurance costs and to build models that can predict charges for new patients.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


## 2. Data Exploration and Preparation


In [ ]:
### 2.1 Explore data patterns and distributions


In [7]:
df = pd.read_csv("../../data/insurance.csv")
df.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## 3. Feature Selection and Justification


### 3.1 Choose features and target

The target variable for this regression problem is **charges**, because the entire purpose of the dataset is to predict medical insurance costs.

I selected the following features: age, bmi, children, sex, smoker, region, bmi_over_30, and the engineering feature age_smoker_interaction. These variables likely influence medical costs, and several (especially smoking status and BMI) have strong known correlations with healthcare spending.

The one-hot encoded version of the dataset (df_encoded) ensures all categorical variables are converted properly for regression.


In [ ]:
# Define X (all columns except the target)
X = df_encoded.drop("charges", axis=1)

# Define y (target)
y = df_encoded["charges"]

X.head(), y.head()
